# Déjame escuchar la música

# Contenido <a id='back'></a>

* [Introducción](#intro)
* [Etapa 1. Descripción de los datos](#data_review)
    * [Conclusiones](#data_review_conclusions)
* [Etapa 2. Preprocesamiento de datos](#data_preprocessing)
    * [2.1 Estilo del encabezado](#header_style)
    * [2.2 Valores ausentes](#missing_values)
    * [2.3 Duplicados](#duplicates)
    * [2.4 Conclusiones](#data_preprocessing_conclusions)
* [Etapa 3. Prueba de hipótesis](#hypothesis)
    * [3.1 Hipótesis 1: actividad de los usuarios y las usuarias en las dos ciudades](#activity)
* [Conclusiones](#end)

## Introducción <a id='intro'></a>
En este proyecto, compararás las preferencias musicales de las ciudades de Springfield y Shelbyville. Estudiarás datos reales de transmisión de música online para probar la hipótesis a continuación y comparar el comportamiento de los usuarios y las usuarias de estas dos ciudades.

### Objetivo:
Prueba la hipótesis:
1. La actividad de los usuarios y las usuarias difiere según el día de la semana y dependiendo de la ciudad.


### Etapas
Los datos del comportamiento del usuario se almacenan en el archivo `/datasets/music_project_en.csv`. No hay ninguna información sobre la calidad de los datos, así que necesitarás examinarlos antes de probar la hipótesis.

Primero, evaluarás la calidad de los datos y verás si los problemas son significativos. Entonces, durante el preprocesamiento de datos, tomarás en cuenta los problemas más críticos.

Tu proyecto consistirá en tres etapas:
 1. Descripción de los datos.
 2. Preprocesamiento de datos.
 3. Prueba de hipótesis.








[Volver a Contenidos](#back)

## Etapa 1. Descripción de los datos <a id='data_review'></a>

Abrir los datos y examínarlos.

In [1]:
# Importar pandas
import pandas as pd

Leer el archivo `music_project_en.csv` de la carpeta `/datasets/` y guárdalo en la variable `df`:

In [2]:
df = pd.read_csv('/datasets/music_project_en.csv')

Muestra las 10 primeras filas de la tabla:

In [3]:
df.head(10)

,userID,Track,artist,genre,City,time,Day
0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Shelbyville,20:28:33,Wednesday
1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Springfield,14:07:09,Friday
2,20EC38,Funiculì funiculà,Mario Lanza,pop,Shelbyville,20:58:07,Wednesday
3,A3DD03C9,Dragons in the Sunset,Fire + Ice,folk,Shelbyville,08:37:09,Monday
4,E2DC1FAE,Soul People,Space Echo,dance,Springfield,08:34:34,Monday
5,842029A1,Chains,Obladaet,rusrap,Shelbyville,13:09:41,Friday
6,4CB90AA5,True,Roman Messer,dance,Springfield,13:00:07,Wednesday
7,F03E1C1F,Feeling This Way,Polina Griffith,dance,Springfield,20:47:49,Wednesday
8,8FA1D3BE,L’estate,Julia Dalia,ruspop,Springfield,09:17:40,Friday
9,E772D5C0,Pessimist,NaN,dance,Shelbyville,21:20:49,Wednesday


Obtener la información general sobre nuestros datos

In [4]:
print(df.info())
print('---------------')
print(df.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65079 entries, 0 to 65078
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0     userID  65079 non-null  object
 1   Track     63736 non-null  object
 2   artist    57512 non-null  object
 3   genre     63881 non-null  object
 4     City    65079 non-null  object
 5   time      65079 non-null  object
 6   Day       65079 non-null  object
dtypes: object(7)
memory usage: 3.5+ MB
None
---------------
          userID  Track     artist  genre       City        time     Day
count      65079  63736      57512  63881        65079     65079   65079
unique     41748  39666      37806    268            2     20392       3
top     A8AE9169  Brand  Kartvelli    pop  Springfield  21:51:22  Friday
freq          76    136        136   8850        45360        14   23149


Estas son nuestras observaciones sobre la tabla. Contiene siete columnas. Almacenan los mismos tipos de datos: `object`.

Según la documentación:
- `' userID'`: identificador del usuario o la usuaria;
- `'Track'`: título de la canción;
- `'artist'`: nombre del artista;
- `'genre'`: género de la pista;
- `'City'`: ciudad del usuario o la usuaria;
- `'time'`: la hora exacta en la que se reprodujo la canción;
- `'Day'`: día de la semana.

Podemos ver tres problemas con el estilo en los encabezados de la tabla:
1. Algunos encabezados están en mayúsculas, otros en minúsculas.
2. Hay espacios en algunos encabezados.
3. Los nombres de las columnas son poco descriptivos sobre la informacion que entrega cada una y en el caso  de userID no se respeta el estilo `snake_case`.




- Se está utilizando solo datos de tipo Objectc que en otro entorno fuera del DF podria diferenciarse entre datos de tipo DateTime/Date y String/Texto.

- Podriamos mejorar y especificar mejor en el nombre de la columna la informacion que entrega o podemos consultar la documentacion de la tabla.

- Para confirmar o negar la hipotesis habra que agrupar los datos por dia de la semana y quizas sea bueno mostrar solo los más relevantes que incluiria: la cuidad, el dia de la semana y el user_id.

- Existen datos nulos en la tabla y tambien duplicados, pero para efectos de este análisis los valores nulos al no estar en las columnas, 'dia', 'ciudad, o 'user_id', no tienen mayor relevancia. En cambio los valores duplicados podrian probocar errores en el analisis

[Volver a Contenidos](#back)

## Etapa 2. Preprocesamiento de datos <a id='data_preprocessing'></a>

El objetivo aquí es preparar los datos para que sean analizados.
El primer paso es resolver cualquier problema con los encabezados. Luego podemos avanzar a los valores ausentes y duplicados. Empecemos.

Corregir el formato en los encabezados de la tabla.


### Estilo del encabezado <a id='header_style'></a>
Muestra los encabezados de la tabla (los nombres de las columnas):

In [5]:
df.columns

Index(['  userID', 'Track', 'artist', 'genre', '  City  ', 'time', 'Day'], dtype='object')

Cambiar los encabezados de la tabla de acuerdo con las reglas del buen estilo:
* Todos los caracteres deben ser minúsculas.
* Elimina los espacios.
* Si el nombre tiene varias palabras, utiliza `snake_case`.

Utilizando el ciclo for se itera sobre las columnas para modificar sus nombres a  minusculas

In [6]:
# Bucle en los encabezados poniendo todo en minúsculas
new_names = []
for column in df.columns :
    lower_name = column.lower()
    new_names.append(lower_name)
df.columns =  new_names

Ahora, utilizando el mismo método, se elimina los espacios al principio y al final de los nombres de las columnas e imprime los nombres de las columnas nuevamente:

In [7]:
# Bucle en los encabezados eliminando los espacios
new_names = []  
for column in df.columns:
    strip_name = column.strip()
    non_space = strip_name.replace(' ','_')
    new_names.append(non_space)
df.columns = new_names

Necesitamos aplicar la regla de snake_case a la columna `userid`. Debe ser `user_id`.

In [8]:
# Cambiar el nombre de la columna "userid"
new_name = {'userid' : 'user_id'}
df.rename(columns = new_name, inplace = True)

Comprueba el resultado. Muestra los encabezados una vez más:

In [8]:
# Comprobar el resultado: la lista de encabezados
df.columns

Index(['userid', 'track', 'artist', 'genre', 'city', 'time', 'day'], dtype='object')

[Volver a Contenidos](#back)

### Valores ausentes <a id='missing_values'></a>
 Encontrar el número de valores ausentes en la tabla.

In [9]:
# Calcular el número de valores ausentes
df.isna().sum()

userid       0
track     1343
artist    7567
genre     1198
city         0
time         0
day          0
dtype: int64

No todos los valores ausentes afectan a la investigación. Por ejemplo, los valores ausentes en `track` y `artist` no son cruciales. Simplemente puedes reemplazarlos con valores predeterminados como el string `'unknown'` (desconocido).

Pero los valores ausentes en `'genre'` pueden afectar la comparación entre las preferencias musicales de Springfield y Shelbyville. En la vida real, sería útil saber las razones por las cuales hay datos ausentes e intentar recuperarlos. Pero no tenemos esa oportunidad en este proyecto. Así que tendrás que:
* rellenar estos valores ausentes con un valor predeterminado;
* evaluar cuánto podrían afectar los valores ausentes a tus cómputos;

Reemplazar los valores ausentes en las columnas `'track'`, `'artist'` y `'genre'` con el string `'unknown'`. Como mostramos anteriormente en las lecciones, la mejor forma de hacerlo es crear una lista que almacene los nombres de las columnas donde se necesita el reemplazo. Luego, utiliza esta lista e itera sobre las columnas donde se necesita el reemplazo haciendo el propio reemplazo.

In [10]:
# Bucle en los encabezados reemplazando los valores ausentes con 'unknown'
columns_to_replace = ['track', 'artist' , 'genre']
for column in columns_to_replace:
    df[column].fillna('unknown', inplace=True)

Comprobar el resultado para asegurarse de que después del reemplazo no haya valores ausentes en el conjunto de datos. Para hacer esto, contar los valores ausentes nuevamente.

In [11]:
# Contar valores ausentes
df.isnull().sum()

userid    0
track     0
artist    0
genre     0
city      0
time      0
day       0
dtype: int64

[Volver a Contenidos](#back)

### Duplicados <a id='duplicates'></a>
Encontrar el número de duplicados explícitos en la tabla.

In [12]:
# Contar duplicados explícitos
df.duplicated().sum()


3826

Ahora, eliminar todos los duplicados. 

In [13]:
# Eliminar duplicados explícitos
df.drop_duplicates(inplace= True)

Comprobemos ahora si eliminamos con éxito todos los duplicados. Contar los duplicados explícitos una vez más para asegurarte de haberlos eliminado todos:

In [14]:
# Comprobar de nuevo si hay duplicados
df.duplicated().sum()

0

Ahora queremos deshacernos de los duplicados implícitos en la columna `genre`. Por ejemplo, el nombre de un género se puede escribir de varias formas. Dichos errores también pueden afectar al resultado.

Para hacerlo, primero mostremos una lista de nombres de género únicos, ordenados en orden alfabético. Para ello:
* Extrae la columna `genre` del DataFrame.
* Llama al método que devolverá todos los valores únicos en la columna extraída.


In [17]:
# Inspeccionar los nombres de géneros únicos
genres = df['genre'].unique()
genres.sort()
n_genres = df['genre'].nunique()
print(n_genres,'\n', genres)

269 
 ['acid' 'acoustic' 'action' 'adult' 'africa' 'afrikaans' 'alternative'
 'ambient' 'americana' 'animated' 'anime' 'arabesk' 'arabic' 'arena'
 'argentinetango' 'art' 'audiobook' 'avantgarde' 'axé' 'baile' 'balkan'
 'beats' 'bigroom' 'black' 'bluegrass' 'blues' 'bollywood' 'bossa'
 'brazilian' 'breakbeat' 'breaks' 'broadway' 'cantautori' 'cantopop'
 'canzone' 'caribbean' 'caucasian' 'celtic' 'chamber' 'children' 'chill'
 'chinese' 'choral' 'christian' 'christmas' 'classical' 'classicmetal'
 'club' 'colombian' 'comedy' 'conjazz' 'contemporary' 'country' 'cuban'
 'dance' 'dancehall' 'dancepop' 'dark' 'death' 'deep' 'deutschrock'
 'deutschspr' 'dirty' 'disco' 'dnb' 'documentary' 'downbeat' 'downtempo'
 'drum' 'dub' 'dubstep' 'eastern' 'easy' 'electronic' 'electropop' 'emo'
 'entehno' 'epicmetal' 'estrada' 'ethnic' 'eurofolk' 'european'
 'experimental' 'extrememetal' 'fado' 'film' 'fitness' 'flamenco' 'folk'
 'folklore' 'folkmetal' 'folkrock' 'folktronica' 'forró' 'frankreich'
 'französ

Buscar en la lista para encontrar duplicados implícitos del género `hiphop`. Estos pueden ser nombres escritos incorrectamente o nombres alternativos para el mismo género.

Verás los siguientes duplicados implícitos:
* `hip`
* `hop`
* `hip-hop`

Para deshacerte de ellos, crea una función llamada `replace_wrong_genres()` con dos parámetros:
* `wrong_genres=`: esta es una lista que contiene todos los valores que necesitas reemplazar.
* `correct_genre=`: este es un string que vas a utilizar como reemplazo.

Como resultado, la función debería corregir los nombres en la columna `'genre'` de la tabla `df`, es decir, remplazar cada valor de la lista `wrong_genres` por el valor en `correct_genre`.

Dentro del cuerpo de la función, utiliza un bucle `'for'` para iterar sobre la lista de géneros incorrectos, extrae la columna `'genre'` y aplica el método `replace` para hacer correcciones.

In [17]:
# Función para reemplazar duplicados implícitos

def replace_wrong_genres(dataFrame, column, wrong_values, right_value):
    for duplicate in wrong_values : 
        dataFrame[column].replace(duplicate, right_value, inplace = True)
    return dataFrame
wrong_genres = ['hip', 'hop', 'hip-hop']
correct_genre='hiphop'

Ahora, llama a `replace_wrong_genres()` y pásale tales argumentos para que retire los duplicados implícitos (`hip`, `hop` y `hip-hop`) y los reemplace por `hiphop`:

In [18]:
# Eliminar duplicados implícitos
df= replace_wrong_genres(df, 'genre', wrong_genres, correct_genre)

Asegúrate de que los nombres duplicados han sido eliminados. Muestra la lista de valores únicos de la columna `'genre'` una vez más:

In [19]:
# Comprobación de duplicados implícitos
genres = df['genre'].unique()
n_genres = df['genre'].nunique()
print(n_genres,'\n', genres)



266 
 ['rock' 'pop' 'folk' 'dance' 'rusrap' 'ruspop' 'world' 'electronic'
 'unknown' 'alternative' 'children' 'rnb' 'hiphop' 'jazz' 'postrock'
 'latin' 'classical' 'metal' 'reggae' 'triphop' 'blues' 'instrumental'
 'rusrock' 'dnb' 'türk' 'post' 'country' 'psychedelic' 'conjazz' 'indie'
 'posthardcore' 'local' 'avantgarde' 'punk' 'videogame' 'techno' 'house'
 'christmas' 'melodic' 'caucasian' 'reggaeton' 'soundtrack' 'singer' 'ska'
 'salsa' 'ambient' 'film' 'western' 'rap' 'beats' "hard'n'heavy"
 'progmetal' 'minimal' 'tropical' 'contemporary' 'new' 'soul' 'holiday'
 'german' 'jpop' 'spiritual' 'urban' 'gospel' 'nujazz' 'folkmetal'
 'trance' 'miscellaneous' 'anime' 'hardcore' 'progressive' 'korean'
 'numetal' 'vocal' 'estrada' 'tango' 'loungeelectronic' 'classicmetal'
 'dubstep' 'club' 'deep' 'southern' 'black' 'folkrock' 'fitness' 'french'
 'disco' 'religious' 'drum' 'extrememetal' 'türkçe' 'experimental' 'easy'
 'metalcore' 'modern' 'argentinetango' 'old' 'swing' 'breaks' 'eurofolk'
 

[Volver a Contenidos](#back)

### Observaciones <a id='data_preprocessing_conclusions'></a>

- Primero, para comprobar cuantos diferentes generos habia inicialmente utilicé el metodo "nunique", que devolvio como resultado 269 valores unicos en esa columna.
- Para eliminar los duplicados señalados en el ejercicio creé una la funcion 'replace_wrong_genres' que toma como parametros: el DataFrame, la columna en la que se reemplazaran los valores, los potenciales nombres incorrectos y los nombres correctos.
- La función recorre la columna 'genre' en el DataFrame señalado en busca de los valores en la lista del parametro  'wrong_values' y por cada coincidencia encontrada, reemplaza su valor con el del parametro 'right_value'
- Creé dos nuevas variables; una lista con los duplicados parciales y un string con el nombre por el que deben reemplazarce los valores incorrectos.
- Por ultimo llame a la funcion 'replace_wrong_genres' y complete los parametros con los datos del dataFrame que estoy usando.
- Se eliminaron 3 valores duplicados parciales, seria más facíl analizar este arreglo en busca de duplicados parciales en una lista ordenada alfabeticamente o si se pudiera usar un metodo para buscar una secuencia de valores especificos dentro de cada string, tambien consideré por un momento que los generos latin y latino, podrian ser duplicados parciales, pero como usuaria de spotify, sé que hay pequeñas diferencias entre ambas categorias

[Volver a Contenidos](#back)

## Etapa 3. Prueba de hipótesis <a id='hypothesis'></a>

### Hipótesis: comparar el comportamiento del usuario o la usuaria en las dos ciudades <a id='activity'></a>

La hipótesis afirma que existen diferencias en la forma en que los usuarios y las usuarias de Springfield y Shelbyville consumen música. Para comprobar esto, usa los datos de tres días de la semana: lunes, miércoles y viernes.

* Agrupa a los usuarios y las usuarias por ciudad.
* Compara el número de canciones que cada grupo reprodujo el lunes, el miércoles y el viernes.


Realiza cada cálculo por separado.

El primer paso es evaluar la actividad del usuario en cada ciudad. Recuerda las etapas dividir-aplicar-combinar de las que hablamos anteriormente en la lección. Tu objetivo ahora es agrupar los datos por ciudad, aplicar el método apropiado para contar durante la etapa de aplicación y luego encontrar la cantidad de canciones reproducidas en cada grupo especificando la columna para obtener el recuento.

A continuación se muestra un ejemplo de cómo debería verse el resultado final:
`df.groupby(by='....')['column'].method()`Realiza cada cálculo por separado.

Para evaluar la actividad de los usuarios y las usuarias en cada ciudad, agrupa los datos por ciudad y encuentra la cantidad de canciones reproducidas en cada grupo.



In [20]:
# Contar las canciones reproducidas en cada ciudad
df.groupby(by = 'city')['day'].count()

city
Shelbyville    18512
Springfield    42741
Name: day, dtype: int64

En Springfield se reproduce más del doble de canciones que en Shelbyville. 
Podria deberse a que hay mayor poblacion en Springfield que en Shelbyville pero para corroborar esta hipotesis nos falta informacion demografica de ambas ciudades y para el caso no es relevante 

Ahora agrupemos los datos por día de la semana y encontremos el número de canciones reproducidas el lunes, miércoles y viernes. Utiliza el mismo método que antes, pero ahora necesitamos una agrupación diferente.

In [21]:
# Calcular las canciones reproducidas en cada uno de los tres días
df.groupby(by = 'day')['city'].count()

day
Friday       21840
Monday       21354
Wednesday    18059
Name: city, dtype: int64

La diferencia es mayor entre las reproducciones por ciudad que por dia de la semana, aunque los viernes y lunes se reproducen más canciones que el miercoles

Crea la función `number_tracks()` para calcular el número de canciones reproducidas en un determinado día **y** ciudad. La función debe aceptar dos parámetros:

- `day`: un día de la semana para filtrar. Por ejemplo, `'Monday'` (lunes).
- `city`: una ciudad para filtrar. Por ejemplo, `'Springfield'`.

Dentro de la función, aplicarás un filtrado consecutivo con indexación lógica.

Primero filtra los datos por día y luego filtra la tabla resultante por ciudad.

Después de filtrar los datos por dos criterios, cuenta el número de valores de la columna 'user_id' en la tabla resultante. Este recuento representa el número de entradas que estás buscando. Guarda el resultado en una nueva variable y devuélvelo desde la función.

In [22]:
# Declara la función number_tracks() con dos parámetros: day= y city=.
def number_tracks(day, city) :
    # Almacena las filas del DataFrame donde el valor en la columna 'day' es igual al parámetro day=
    day_df = df[df['day'] == day]
    #Filtra las filas donde el valor en la columna 'city' es igual al parámetro city=x
    city_df = day_df[day_df['city'] == city]
    # Extrae la columna 'user_id' de la tabla filtrada y aplica el método count()
    df_filtered = city_df['user_id'].count()
    # Devolve el número de valores de la columna 'user_id'
    return df_filtered

Llama a `number_tracks()` seis veces, cambiando los valores de los parámetros para que recuperes los datos de ambas ciudades para cada uno de los tres días.

In [23]:
# El número de canciones reproducidas en Springfield el lunes
mon_spring = number_tracks('Monday', 'Springfield' )
mon_spring

15740

In [24]:
# El número de canciones reproducidas en Shelbyville el lunes
mon_shell = number_tracks('Monday', 'Shelbyville' )

In [25]:
# El número de canciones reproducidas en Springfield el miércoles
wend_spring = number_tracks('Wednesday', 'Springfield' )

In [26]:
# El número de canciones reproducidas en Shelbyville el miércoles
wend_shell = number_tracks('Wednesday', 'Shelbyville' )

In [27]:
# El número de canciones reproducidas en Springfield el viernes
fry_spring = number_tracks('Friday', 'Springfield' )

In [28]:
# El número de canciones reproducidas en Shelbyville el viernes
fry_shell = number_tracks('Friday', 'Shelbyville' )

In [29]:
# Crea una tabla con los valores filtrados
day_of_week = [['Monday', mon_spring, mon_shell ],
               ['Wendesday', wend_spring, wend_shell ],
               ['Friday', fry_spring, fry_shell]]
col_cities = ['Day','Springfield', 'Shelbyville' ]
final_df = pd.DataFrame(data=day_of_week, columns=col_cities)
final_df

,Day,Springfield,Shelbyville
0,Monday,15740,5614
1,Wendesday,11056,7003
2,Friday,15945,5895


**Conclusiones**

La hipotesis parece ser correcta, ya que hay una clara diferencia entre los datos de Springfiled y Shelbyville. En Springfield más usuarios reproducen musica los dias lunes y viernes, y son  considerablemente menos activos los dias miercoles, mientras que en Shelbyville ocurre lo contrario ya que el miércoles es el día con mas reproducciones en esa ciudad.

[Volver a Contenidos](#back)

# Conclusiones <a id='end'></a>

Springfiled y Shelbyville siempre han sido rivales por lo que es esperable que tengan comportamientos opuestos aunque en ambas ciudades el comportamiento de los usuarios es similar los dias lunes y viernes y muy diferente el miércoles.
Podira deducir también que en Shelbyville los usuarios escuchan menos musica, pero podria deberse a que hay menos poblacion en esa ciudad. Por ahora no contamos con informacion demografica de la zona como para llegar a dicha conclusion, aunque supongo que seria útil si queremos comparar el comportamiento de ambas ciudades calculando el porcentaje de la población que usa la aplicacion y saber si la diferencia del monto total de reproducciones por ciudad se debe a una menor poblacion o que a los usuarios no les gusta mucho la aplicación.

### Nota
En proyectos de investigación reales, la prueba de hipótesis estadística es más precisa y cuantitativa. También ten en cuenta que no siempre se pueden sacar conclusiones sobre una ciudad entera a partir de datos de una sola fuente.

Aprenderás más sobre la prueba de hipótesis en el sprint de análisis estadístico de datos.

[Volver a Contenidos](#back)